<a href="https://colab.research.google.com/github/seyeonJeong/LikeOrDislike/blob/main/LikeOrDislike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 29.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm
import time

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

train_data = pd.read_table('ratings.txt')

train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)


False


<ipython-input-2-32d0b65ff158>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
100%|██████████| 199992/199992 [17:46<00:00, 187.45it/s]


In [4]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)


In [5]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape


(16477, 100)

In [104]:
okt = Okt()
def tokenize(doc):
    return okt.morphs(doc, stem=True)

print("좋아하는 장르를 입력하세요!")
positive1 = input()
print("싫어하는 장르를 입력하세요!")
negative1 = input()

positive = "장르 "+positive1
negative = negative1
print(model.wv.most_similar(positive=tokenize(positive),
                               negative=tokenize(negative), topn=5))

좋아하는 장르를 입력하세요!
히어로
싫어하는 장르를 입력하세요!
느와르
[('코미디', 0.7673978805541992), ('스릴러', 0.737483024597168), ('판타지', 0.7193078398704529), ('멜로', 0.7038779854774475), ('공포', 0.6646823883056641)]
